In [1]:
# Import functions #
import argparse,datetime,os,sys,time

import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot

In [2]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd

# path to input of local data from MRIQC on your own dataset 
group_file = os.path.join(here,'test_data', 'group2_bold.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   SNR, TSNR, DVAR, FD,
   FWHM, Tesla, gsr_x, gsr_y, TE, TR
   NOTE: Only working as *and* right now!
"""
filter_list = ['TR > 2.0','FD < .3']

# IQM variables to visualize
IQM_to_plot = ['fwhm_avg','fber','fd_mean','gsr_x']

In [3]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape

(142, 45)

In [4]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
filtered_apidf = filterIQM(apidf,filter_list)
# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)

(1000, 77)
(203, 77)


In [7]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape



(345, 78)

### Color coding
Families of IQMs are color-coded. In each plot, the API data is shown in dark gray, and the user's data is shown in the appropriate color. 
- spatial IQMs: gold
- temporal IQMs: orange
- noise IQMs: red
- motion IQMs: green
- artifact IQMs: light blue
- descriptive IQMs: dark blue
- other IQMs: purple

In [6]:
# Visualization # 

# Create a violin plot of each MRIQC metric, comparing user level-data to API data 
# Also will print out a short description of the measure and what the appropriate range/values should be 
# Input the merged dataframe including the API and user data
# Optional arguments include a list of specific variables that the user wants displayed 
# If no optional arguments included, will plot all measures 

v = make_vio_plot(vis_ready_df,IQM_to_plot,"",outliers=True)

widgets.VBox([v[0],v[1]])

Loading in dataframe...
Loading variables: ['fwhm_avg', 'fber', 'fd_mean', 'gsr_x']
Loading in data descriptors...


TypeError: '<' not supported between instances of 'str' and 'int'

TypeError: can't multiply sequence by non-int of type 'float'